In [1]:
# pip install tensorflow
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix

2025-05-12 09:19:04.897082: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 09:19:05.014722: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-12 09:19:05.112290: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747030745.227877    6580 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747030745.261795    6580 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747030745.461563    6580 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
# Organize dataset into folders
source_folder = "/home/innodems-dan/Documents/jupyter-folder/project/maize_leaf_disease/data"
# destination_folder = source_folder
# classes = ["blight", "Common_Rust", "Gray_Leaf_Spot", "Healthy"]

# for cls in classes:
#     os.makedirs(os.path.join(destination_folder, cls), exist_ok=True)

# for filename in os.listdir(source_folder):
#     if filename.endswith((".jpg", ".png", ".jpeg")):
#         for cls in classes:
#             if cls.lower().replace("_", "") in filename.lower().replace("_", ""):
#                 src = os.path.join(source_folder, filename)
#                 dst = os.path.join(destination_folder, cls, filename)
#                 shutil.move(src, dst)
#                 break
# print("Files organized into class folders.")

In [3]:
#Spliting the dataset into 80% training dataset
# train_dataset = image_dataset_from_directory(
#   data_dir,
#   validation_split=0.2,
#   subset="training",
#   seed=123,
#   image_size=(img_height, img_width),
#   batch_size=batch_size)
train_ds = tf.keras.utils.image_dataset_from_directory(
    source_folder,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224, 224),
    batch_size=32
)

Found 4198 files belonging to 4 classes.
Using 3359 files for training.


2025-05-12 09:19:29.166094: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# train_ds = tf.keras.utils.image_dataset_from_directory(
#     source_folder,
#     validation_split=0.2,
#     subset="training",
#     seed=123,
#     image_size=(224, 224),
#     batch_size=32
# )

# val_ds = tf.keras.utils.image_dataset_from_directory(
#     source_folder,
#     validation_split=0.2,
#     subset="validation",
#     seed=123,
#     image_size=(224, 224),
#     batch_size=32
# )

class_names = train_ds.class_names
print("Classes:", class_names)

Classes: ['Blight', 'Common_Rust', 'Gray_Leaf_Spot', 'Healthy']


In [5]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

model = tf.keras.Sequential([
    data_augmentation,
    layers.Rescaling(1./255, input_shape=(224, 224, 3)),
    layers.Conv2D(32, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(64, 3, activation="relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(len(class_names), activation="softmax")
])

model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

/home/innodems-dan/.local/share/pipx/venvs/jupyterlab/lib/python3.12/site-packages/keras/src/layers/preprocessing/tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5


2025-05-06 09:46:14.682568: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2025-05-06 09:46:14.821786: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2025-05-06 09:46:15.462004: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2025-05-06 09:46:15.635996: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.
2025-05-06 09:46:15.836655: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 95551488 exceeds 10% of free system memory.


105/105 ━━━━━━━━━━━━━━━━━━━━ 326s 3s/step - accuracy: 0.6119 - loss: 1.5346 - val_accuracy: 0.8200 - val_loss: 0.4403
Epoch 2/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 276s 3s/step - accuracy: 0.8326 - loss: 0.3740 - val_accuracy: 0.8462 - val_loss: 0.3633
Epoch 3/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 285s 3s/step - accuracy: 0.8450 - loss: 0.3504 - val_accuracy: 0.8403 - val_loss: 0.3719
Epoch 4/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 250s 2s/step - accuracy: 0.8520 - loss: 0.3496 - val_accuracy: 0.8617 - val_loss: 0.3147
Epoch 5/5
105/105 ━━━━━━━━━━━━━━━━━━━━ 234s 2s/step - accuracy: 0.8633 - loss: 0.3486 - val_accuracy: 0.8713 - val_loss: 0.3119


In [12]:
model.evaluate(val_ds)

# Predict a new image (example)
img_path = "/home/innodems-dan/scikit_learn_data/testing-2.jpg"  # Replace with actual path
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
predicted_class = class_names[np.argmax(predictions)]
print("Predicted class:", predicted_class)

27/27 ━━━━━━━━━━━━━━━━━━━━ 8s 306ms/step - accuracy: 0.8798 - loss: 0.3004 
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted class: Gray_Leaf_Spot
